**Run on Google Colab (Quickstart)**

```bash
! git clone --branch main --single-branch https://github.com/sbaaihamza/scrapping-lib.git
%cd scrapping-lib
! pip install -e ".[browser,dev]"
! playwright install
# Preferred (installs OS deps automatically on supported distros):
! playwright install --with-deps chromium
# If needed (manual deps fallback):
! apt-get update
! apt-get install -y libxcomposite1 libxcursor1 libgtk-3-0 libatk1.0-0 libcairo2 libgdk-pixbuf2.0-0
%cd /content/scrapping-lib/notebooks
```

*Note: Playwright has both sync and async APIs. These notebooks are designed to be async-safe for Jupyter/Colab. If you encounter OS dependency issues, use the `playwright install --with-deps chromium` command.*



# Engine In-Depth: Hybrid Engine Cases

This notebook demonstrates the `hybrid` engine, combining the speed of HTTP discovery with the accuracy of Browser detail extraction.

### Learning Outcomes
*   Understand when a **Hybrid approach** is justified (High speed vs. High accuracy).
*   Configure **Fallback Policies** based on content quality.
*   Implement **Diagnostics-driven Fallbacks** (e.g., if HTTP reports `js_required`).
*   Interpret **Multi-engine Traces** for full observability.

## 0) Setup

In [ ]:
import json
import os
import sys
from pathlib import Path

from scrapping.diagnostics.classifiers import DiagnosisLabel
from scrapping.engines.hybrid import HybridEngine, HybridEngineOptions

REPO_ROOT = Path.cwd().parent
sys.path.append(str(REPO_ROOT))
os.chdir(str(REPO_ROOT))

ONLINE = os.getenv('ONLINE', '0') == '1'
RESULTS_DIR = Path('results_notebook_hybrid')
print(f'Online mode: {ONLINE}')

## 1) Why Hybrid?

We use **HTTP** for fast link discovery (Listing pages) and **Browser** for accurate content extraction (Detail pages). This minimizes resource consumption while ensuring data quality.

In [ ]:
engine = HybridEngine()

url_list = 'https://quotes.toscrape.com/'
url_detail = 'https://quotes.toscrape.com/author/Albert-Einstein/'

if ONLINE:
    res_list = engine.get(url_list)
    print(f"Listing (HTTP) OK: {res_list.ok}")
    
    res_det = engine.get_rendered(url_detail)
    print(f"Detail (Browser) OK: {res_det.ok}")
else:
    print("Offline: Engine initialized. Use fixtures for multi-engine simulation.")

## 2) Fallback Behavior

The Hybrid engine can automatically fallback to Browser if HTTP fails or returns low-quality content (e.g., body length < 500 chars).

In [ ]:
opts = HybridEngineOptions(
    fallback_to_browser=True, 
    min_text_len=500
)
engine_fb = HybridEngine(options=opts)
print(f"Hybrid configured with fallback. Min text len: {opts.min_text_len}")

## 3) Diagnostics-Driven Fallback

Modern scrapers use signals to decide when to escalate. 
*   If HTTP reports `js_required_or_missing_content` -> **Switch to Browser**.
*   If `challenge_detected` -> **Stop immediately**.

In [ ]:
from scrapping.diagnostics.classifiers import diagnose_http_response


def smart_get(url, engine):
    res = engine.http.get(url)
    diag = diagnose_http_response(res.status_code, res.response_meta.headers, res.text)
    
    if diag.label == DiagnosisLabel.JS_REQUIRED_OR_MISSING_CONTENT:
        print(f"Escalating {url} to Browser engine...")
        return engine.browser.get(url)
    elif diag.label == DiagnosisLabel.CHALLENGE_DETECTED:
        print(f"Challenge detected on {url}. Stopping.")
        return res
    
    return res

print("Smart fetching logic defined based on diagnostic signals.")

## 4) Observability: Multi-engine Traces

Results from a Hybrid engine include a combined trace showing exactly which engine was used and why fallbacks were triggered.

In [ ]:
if ONLINE:
    res = engine_fb.get('https://example.com')
    print(f"Engine Trace: {json.dumps(res.engine_trace, indent=2)}")
else:
    print("Offline: Trace would show the sequence of engine attempts.")

### Verification Checklist
- [ ] HTTP discovery is used for listing pages to save resources.
- [ ] Browser rendering is used for detail pages or as a validated fallback.
- [ ] Diagnostics correctly trigger engine escalation.
- [ ] Combined traces provide full visibility into the execution path.